In [94]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider, VBox, Output
import IPython.display as display
from scipy.integrate import solve_ivp

y0 = [0, 0.0, 0.3, 0.0] # warunki początkowe: x1, v1, x2, v2

timebases = [20, 300, 100]

# równania różniczkowe oscylatorów
def coupled_oscillators(t, y, k, ks, c):
    m1, m2 = 1.0, 1.0  # masy obiektów
    L = 1  # stała długość sprężyny
    x1, v1, x2, v2 = y
    
    # Równania ruchu
    dx1dt = v1
    dv1dt = (-k*x1 + ks*(x2 - x1 - L) - c*v1) / m1
    dx2dt = v2
    dv2dt = (-k*(x2 - L)+ ks*(x1 - x2 + L) - c*v2) / m2

    return [dx1dt, dv1dt, dx2dt, dv2dt]

def solve_ode_n(k, ks, c, y0, n):
    t_span = (0, n)
    t_eval = np.linspace(0, n, 50*n)
    sol = solve_ivp(coupled_oscillators, t_span, y0, args=(k, ks, c), t_eval=t_eval)
    return sol.t, sol.y

def update_graph_n(k, ks, c, n, ax):
    ax.clear()
    t, y = solve_ode_n(k, ks, c, y0, n)
    ax.set_xlim(0, n)
    ax.set_ylim(-2, 2)
    ax.set_xlabel('Czas')
    ax.set_ylabel('Przemieszczenie')
    ax.grid()
    line1, = ax.plot(t, y[0], lw=1.5, color='blue', label='Oscylator 1')
    line2, = ax.plot(t, y[2], lw=1, color='red', label='Oscylator 2')
    ax.legend()
    with output:
        display.display(ax.figure)

def update_all(change):
    output.clear_output(wait=True)
    for n in timebases:
        fig, ax = plt.subplots()
        update_graph_n(spring_slider.value, coupling_slider.value, damping_slider.value, n, ax)
    
        
spring_slider = FloatSlider(min=0.1, max=2.0, step=0.01, value=1.5, description='Stała sprężystości')
coupling_slider = FloatSlider(min=0.01, max=0.2, step=0.01, value=0.2, description='Sprzężenie')
damping_slider = FloatSlider(min=0.001, max=0.05 , step=0.001, value=0.05, description='Tłumienie')

ui = VBox([spring_slider, coupling_slider, damping_slider])
output = Output()

spring_slider.observe(update_all, names='value')
coupling_slider.observe(update_all, names='value')
damping_slider.observe(update_all, names='value')

display.display(ui)
display.display(output)

Output()